<a href="https://colab.research.google.com/github/jeguns/Manipulacion-de-datos-con-R-y-Python/blob/main/Python/10%20-%20Res%C3%BAmenes%20estad%C3%ADsticos/U10_Resumenes_Estadisticos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Lectura

In [ ]:
import pandas as pd
datos1 = pd.read_csv('/content/drive/MyDrive/Cursos/Manipulación de datos con R y Python/U10 - DATOS1.csv', encoding='latin1')
datos2 = pd.read_csv('/content/drive/MyDrive/Cursos/Manipulación de datos con R y Python/U10 - DATOS2.csv', encoding='latin1')

## Repaso

In [ ]:
import numpy as np

datos_A = (
    datos1
    .assign(CODIGO=lambda x: x['CONGLOME'].astype(str) + x['VIVIENDA'].astype(str) + x['HOGAR'].astype(str) + x['CODPERSO'].astype(str))
    .loc[:, ['CODIGO', 'DOMINIO', 'P207', 'P208A', 'P209']]
    .assign(
        DOMINIO=lambda x: x['DOMINIO'].map({1: "Costa Norte",
                                            2: "Costa Centro",
                                            3: "Costa Sur",
                                            4: "Sierra Norte",
                                            5: "Sierra Centro",
                                            6: "Sierra Sur",
                                            7: "Selva",
                                            8: "Lima Metropolitana"}),
        SEXO=lambda x: x['P207'].map({'1': "Hombre", '2': "Mujer"}),
        EDAD=lambda x: pd.to_numeric(x['P208A'], errors='coerce'),
        ECIV=lambda x: x['P209'].map({'1': "Conviviente",
                                      '2': "Casado(a)",
                                      '3': "Viudo(a)",
                                      '4': "Divorciado(a)",
                                      '5': "Separado(a)",
                                      '6': "Soltero(a)"}))
    .drop(columns=['P207', 'P208A', 'P209'])
)

datos_A.head()


In [ ]:
datos_A.info()

In [ ]:
datos_B = (
    datos2
    .assign(CODIGO=lambda x: x['CONGLOME'].astype(str) + x['VIVIENDA'].astype(str) + x['HOGAR'].astype(str) + x['CODPERSO'].astype(str))
    .loc[:, ['CODIGO', 'P314A', 'P314D']]
    .rename(columns={'P314A': 'INTERNET', 'P314D': 'FREC_INT'})
    .assign(
        INTERNET=lambda x: x['INTERNET'].map({'1': "Sí", '2': "No"}).astype("category"),
        FREC_INT=lambda x: x['FREC_INT'].map({
            '1': "Una vez al día",
            '2': "Una vez a la semana",
            '3': "Una vez al mes",
            '4': "Cada 2 meses o más"
        }).astype("category")
    )
)

datos_B.head()

In [ ]:
datos = pd.merge(datos_A, datos_B, on="CODIGO", how="inner")
datos.head()

In [ ]:
datos['EDAD'].mean()

In [ ]:
datos['EDAD'].agg(mean='mean')

In [ ]:
print(datos[datos['DOMINIO'] == "Costa Sur"]['EDAD'].mean())
print(datos[datos['DOMINIO'] == "Costa Centro"]['EDAD'].mean())
print(datos[datos['DOMINIO'] == "Costa Norte"]['EDAD'].mean())

In [ ]:
datos_grouped0 = (datos
                 .groupby('DOMINIO', as_index=False)
                 .agg(PromEdad=('EDAD', 'mean')))
datos_grouped0

In [ ]:
datos_grouped1 = (datos
                 .groupby('DOMINIO', as_index=False)
                 .agg(PromEdad=('EDAD', 'mean'))
                 .sort_values(by='PromEdad', ascending=False)
                 .reset_index(drop=True))
datos_grouped1

In [ ]:
datos_grouped2 = (datos
                 .groupby('DOMINIO', as_index=False)
                 .agg(Mediana_Edad=('EDAD', 'median'))
                 .sort_values(by='Mediana_Edad', ascending=False)
                 .reset_index(drop=True))
datos_grouped2

In [ ]:
datos_grouped3 = (datos
                  .groupby('DOMINIO', as_index=False)
                  .agg(P25_Edad     = ('EDAD', lambda x: x.quantile(0.25)),
                       Mediana_Edad = ('EDAD', lambda x: x.quantile(0.5)),
                       P75_Edad     = ('EDAD', lambda x: x.quantile(0.75))))
datos_grouped3

In [ ]:
pip install ydata_profiling > /dev/null 2>&1

In [ ]:
from ydata_profiling import ProfileReport
profile = ProfileReport(datos.iloc[0:100,:], title="Resumen de Datos")
profile.to_notebook_iframe()

In [ ]:
profile.to_file("reporte_datos.html")